In [72]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from tensorflow.keras.datasets import mnist
import tensorflow as tf

### 加载数据

In [73]:
(X_train,y_train),(X_test,y_test) = mnist.load_data()

# 独热编码：概率形式表示数据
# 分类大部分都是概率问题
# 目标值变换成概率
y_train = tf.one_hot(y_train,depth=10)
y_test = tf.one_hot(y_test,depth=10)
# 归一化
X_train = X_train.reshape(60000,784).astype(np.float32)/255.0
X_test = X_test.reshape(10000,784).astype(np.float32)/255.0

data_train = tf.data.Dataset.from_tensor_slices((X_train,y_train))
data_train = data_train.repeat(100).shuffle(2000).batch(256)

data_test = tf.data.Dataset.from_tensor_slices((X_test,y_test))
data_test = data_test.repeat(5).shuffle(3000).batch(1000)

### 声明模型的斜率和截距
### 声明模型和损失
### 声明优化算法SGD

In [74]:
# X -----w + b -----> y
w = tf.Variable(tf.random.normal(shape = [784,10],stddev = 0.1),name = 'weights')
b = tf.Variable(tf.random.normal(shape = [10],stddev = 0.1),name = 'bias')
# 逻辑斯蒂回归模型
def logistic_model(X):
    y_pred = tf.nn.softmax(tf.matmul(X,w) + b)# 软最大，转化成概率
    return y_pred
# 定义损失函数，使用交叉熵
# tf.reduce_mean:平均交叉熵
# 线性回归时：平均二乘法
def cross_entropy(y_pred,y_true):#y_true和y_pred都是多个样本的
    y_pred = tf.clip_by_value(y_pred,clip_value_min=1e-8,clip_value_max=1.0)
    loss = tf.reduce_mean(tf.reduce_sum(tf.multiply(y_true,tf.math.log(1/y_pred)),axis = -1))
    return loss
# 优化方式：随机梯度下降
sgd = tf.optimizers.SGD(learning_rate=0.01)

### 定义优化方法

In [75]:
# 定义优化过程和线性回归，一模一样！！！
def run_optimizer(X_train,y_train):
    with tf.GradientTape() as g:
        y_pred = logistic_model(X_train)
        loss = cross_entropy(y_pred,y_train)
        gradients = g.gradient(loss,[w,b])
    sgd.apply_gradients(zip(gradients,[w,b]))

### 运行优化代码，计算准确率

In [77]:
for i,(X_train,y_train) in enumerate(data_train.take(1000),1):
    run_optimizer(X_train,y_train)
    if i %50 == 0:#计算准确率，测试数据data_test测试数据
        for X_test,y_test in data_test.take(1):#每次和每次取出来的值，不同的
            y_ = logistic_model(X_test).numpy().argmax(axis = 1)
            y_true = y_test.numpy().argmax(axis = 1)
            accuracy = (y_ == y_true).mean()
            print('运行次数：%d。准确率是：%0.4f'%(i,accuracy))

运行次数：50。准确率是：0.7710
运行次数：100。准确率是：0.8090
运行次数：150。准确率是：0.8140
运行次数：200。准确率是：0.8240
运行次数：250。准确率是：0.8060
运行次数：300。准确率是：0.8300
运行次数：350。准确率是：0.8210
运行次数：400。准确率是：0.8230
运行次数：450。准确率是：0.8230
运行次数：500。准确率是：0.8100
运行次数：550。准确率是：0.8380
运行次数：600。准确率是：0.8390
运行次数：650。准确率是：0.8210
运行次数：700。准确率是：0.8420
运行次数：750。准确率是：0.8420
运行次数：800。准确率是：0.8250
运行次数：850。准确率是：0.8300
运行次数：900。准确率是：0.8470
运行次数：950。准确率是：0.8360
运行次数：1000。准确率是：0.8560
